# Test with Resnet50

In [10]:
import os
import glob
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import keras

from keras.applications import ResNet50

from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.layers import BatchNormalization, GlobalAveragePooling2D, SeparableConv2D

from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

from keras.preprocessing.image import ImageDataGenerator


In [5]:
train_data_path = os.path.join('..', 'Dataset', 'CUB_data', 'data', 'train') 
valid_data_path = os.path.join('..', 'Dataset', 'CUB_data', 'data', 'test') 

train_data_agumentation = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2  # 20% for validation
)

val_data_agumentation = ImageDataGenerator(rescale = 1./255)

# load training data
train_generator = train_data_agumentation.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)
validation_generator = val_data_agumentation.flow_from_directory(
    valid_data_path,
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)

Found 4156 images belonging to 2 classes.
Found 893 images belonging to 2 classes.


In [8]:
checkpoint_dir = 'checkpoints_CUB_ResNet50_models'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_filepath = os.path.join(checkpoint_dir,
                                   "model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}_val_loss_{val_loss:.2f}.keras")

checkpoint = ModelCheckpoint(filepath = checkpoint_filepath,
                             monitor = 'val_acc', # 'val_loss',
                             verbose = 0,
                             save_best_only = True,
                             save_weights_only = False,
                             mode = 'max',
                             save_best_only=True)

early = EarlyStopping(monitor = 'val_loss',
                      min_delta = 0,
                      patience = 10,
                      verbose = 0,
                      mode = 'auto')

reduceLR = ReduceLROnPlateau(monitor = "val_loss",
                             factor = 0.1,
                             patience = 5,
                             verbose = 0,
                             mode = "auto",
                             min_delta = 0.0001,
                             cooldown = 0,
                             min_lr = 0.0)

csv_logger = CSVLogger(os.path.join(checkpoint_dir, 'training.log'))

callbacks_list = [checkpoint, early, reduceLR, csv_logger] #, early 


In [11]:
image_size = [224, 224]
num_classes = len(glob.glob(train_data_path+'/*'))

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(224, 224, 3))

for layer in base_model.layers[:140]:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 14982146 (57.15 MB)
Non-trainable params: 8609664 (32.84 MB)
_________________________________________________________________


In [12]:
checkpoint_dir = 'checkpoints_ResNet50_models'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# checkpoint_filepath = os.path.join(checkpoint_dir,
#                                    "model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}_val_loss_{val_loss:.2f}.keras")
# check_point = ModelCheckpoint(filepath=checkpoint_filepath,
#                               monitor="val_acc",
#                               mode="max",
#                               save_best_only=True,
#                               )

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(train_generator, 
                    batch_size=8, 
                    epochs=100, 
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks=[callbacks_list])

# Save the trained model       
save_model_path = os.path.join(checkpoint_dir, 'ResNet50_model.h5')
model.save(save_model_path)
print(f"Model saved as {save_model_path}.")

Epoch 1/100


520/520 [==============================] - 1237s 2s/step - loss: 0.7244 - accuracy: 0.4976 - val_loss: 0.7243 - val_accuracy: 0.4849 - lr: 1.0000e-05
Epoch 2/100
520/520 [==============================] - 1401s 3s/step - loss: 0.7121 - accuracy: 0.5214 - val_loss: 0.7225 - val_accuracy: 0.4703 - lr: 1.0000e-05
Epoch 3/100
506/520 [============================>.] - ETA: 33s - loss: 0.7124 - accuracy: 0.5062

KeyboardInterrupt: 